In [43]:
#export
import glob
import pandas as pd
import numpy as np
import os
pd.set_option('mode.chained_assignment', None)

In [44]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)                                

def removeFile(filename):
    try:
        os.remove(filename)
    except OSError:
        pass   
    
def removeFolder(foldername):
    try:
        shutil.rmtree(foldername)
    except OSError:
        print(f"{foldername} not found")
        pass 

In [45]:
#export
def concatRsubreadDeseq(outdir):
    df_final = pd.DataFrame()
    rsubread_deseq_path = f"{outdir}/rsubread_deseq"
    for folder in glob.glob(f"{rsubread_deseq_path}/rsubread_deseq*"):
        df_g = pd.read_csv(f"{folder}/DESeq2_sense.csv", sep=",")
        df_g.rename(columns={"Unnamed: 0" : "name_latent_state"}, inplace=True)
        df_final = pd.concat([df_final,df_g])
    return df_final

def parseGTF(outdir):
    df_final = pd.DataFrame()
    for f in glob.glob(f"{outdir}/Chrom_out/*/*gtf"): 
#         f = f"{outdir}/gtf/ConvolutionWindow_10_Iterations_40.gtf"
        names = ["chr","source","type","start", "end","dot","strand","dot2","name_latent_state"]
        df = pd.read_csv(f, sep="\t",names=names)
        df["name_latent_state"] = df["name_latent_state"].str.replace('gene_id \"',"")
        df["name_latent_state"] = df["name_latent_state"].str.replace('\";',"")
        df = df[["chr","start", "end","strand","name_latent_state"]]
        df_final = pd.concat([df_final,df])
    return df_final
    
def mergeGTFSubread(df_subread,df_gtf):
    df = pd.merge(df_subread,df_gtf,on="name_latent_state",how="left")
    df["name"] = df["name_latent_state"].str.split('_',expand=True)[0]
    df = df.drop_duplicates(subset=['name_latent_state'])
    df["LATENT_OR_GENE"] = np.where(df['name_latent_state'].str.contains(r'_gene')==True,"GENE","LATENT") 
    return df
    
    

In [46]:
outdir = "NEXTFLOW_MaDDoG/MaDDoG"
df_subread = concatRsubreadDeseq(outdir)
df_gtf = parseGTF(outdir)
df_s = mergeGTFSubread(df_subread,df_gtf)

In [47]:
df_s

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE
0,ZNF596_1,32.845751,0.564329,0.751698,0.750739,0.452810,0.620300,chr8,247340,253590,+,ZNF596,LATENT
1,ZNF596_2,1.956140,-0.718990,1.375784,-0.522604,0.601250,NaN,chr8,253591,256890,+,ZNF596,LATENT
2,ZNF596_3,1.240936,-0.729333,2.496705,-0.292118,0.770196,NaN,chr8,256891,262340,+,ZNF596,LATENT
3,FBXO25_1,148.468020,1.650787,0.619392,2.665175,0.007695,0.031072,chr8,477967,497517,+,FBXO25,LATENT
4,FBXO25_2,31.702551,2.032250,0.520938,3.901139,0.000096,0.000816,chr8,497518,522967,+,FBXO25,LATENT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38145,UCKL1_gene,3779.734537,-1.171326,0.934501,-1.253424,NaN,NaN,chr20,63939829,63956447,-,UCKL1,GENE
38146,ZNF512B_gene,4108.799130,-0.765251,0.731770,-1.045755,0.295674,0.482275,chr20,63956703,63969930,-,ZNF512B,GENE
38147,SAMD10_gene,245.763521,-2.867937,0.821542,-3.490922,0.000481,0.003340,chr20,63974113,63980788,-,SAMD10,GENE
38148,SOX18_gene,176.668734,-0.479675,0.751570,-0.638230,0.523324,0.699315,chr20,64047582,64049639,-,SOX18,GENE


In [68]:
#export
def get_sig_csv(df, padj, log2FoldChange):
    df["significant"] = np.where(df["padj"] < padj, True, False)
    df_final = df[df.significant]  #Keep all True values. (significant)
#     del df_final["significant"]
    df_final = df_final.copy()
    #Treatment vs Control
    
    df_final["T_vs_C_up"] = np.where(df_final["log2FoldChange"] > log2FoldChange, True, False)
    df_up = df_final[df_final.T_vs_C_up]  #Keep all True values. (up)
    del df_up["T_vs_C_up"]

    df_final["T_vs_C_down"] = np.where(df_final["log2FoldChange"] < log2FoldChange, True, False)
    df_do = df_final[df_final.T_vs_C_down]  #Keep all True values. (down)
    del df_do["T_vs_C_up"]
    del df_do["T_vs_C_down"]
    
    df_up["significant_condition"] = "T"
    df_do["significant_condition"] = "C"
    df_sig = pd.concat([df_up,df_do])
    df_sig2 = df_sig[["name_latent_state","significant_condition"]]
    df_all = pd.merge(df,df_sig2,how="outer",on="name_latent_state")    
    df_all["significant_condition"] = df_all["significant_condition"].fillna("NA")
    return df_sig, df_all

def makeBed(df,bedtitle,outdir,Title):
#     df = df[df["significant"]==True]
    df["start2"] = df["start"]
    df["stop2"] = df["end"]
    df["zero"] = "0"        


    Lime = "0,255,0"
    Blue = "0,0,255"
    Red = "255,0,0"    
    df["RGB"] = Blue
    df["RGB"] = np.where(df["significant_condition"]=="T",Lime,df["RGB"])
    df["RGB"] = np.where(df["significant_condition"]=="C",Red,df["RGB"])    
    
    df = df[["chr","start","end","name_latent_state","zero","strand","start2","stop2","RGB"]]    
    
    makeFolders([f"{outdir}/BEDOUT/"])
    bed_out = f"{outdir}/BEDOUT/{Title}_{bedtitle}.bed"         
    open(bed_out, 'w').write('track name="'+Title+'" description="'+Title+'" visibility=2 itemRgb="On"\n')
    df.to_csv(bed_out, sep="\t", index=None, columns=None, header=None)     
    


In [69]:
padj = 0.05
log2FoldChange = 0

In [70]:
#export
# f_dseq = f"{SwitchOutFolder}/initial_Rsubread_DESeq2/DESeq2_sense.csv"

def runALL(outdir):
    
    
    df_subread = concatRsubreadDeseq(outdir)
    df_gtf = parseGTF(outdir)
    df_s = mergeGTFSubread(df_subread,df_gtf) 
    
    # Get sig latent states
    df_sig,df_all = get_sig_csv(df_s, padj, log2FoldChange)    
    df_sig_latent = df_sig[df_sig["LATENT_OR_GENE"]=="LATENT"] 
    
#     # Get all genes that have any sig latent state
    df_sig_latent = df_sig_latent[["name"]]
    df_all = pd.merge(df_all,df_sig_latent,how="inner",on="name") 
    df_all = df_all.drop_duplicates(subset=['name_latent_state'])
    df_gene = df_all[df_all["LATENT_OR_GENE"]=="GENE"] 
    df_latent = df_all[df_all["LATENT_OR_GENE"]=="LATENT"]     

    
    makeBed(df_all,bedtitle="all",outdir=outdir,Title="all")
    
# #     makeSigBed(df_sig,bedtitle="sig",outdir,Title="sig")    

# #     df_latent, df_gene
    return df_sig_latent, df_all, df_gene, df_latent

df_sig_latent, df_all, df_gene, df_latent = runALL(outdir=outdir)

In [71]:
df_all

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE,significant,significant_condition,start2,stop2,zero,RGB
0,FBXO25_1,148.468020,1.650787,0.619392,2.665175,7.694834e-03,0.031072,chr8,477967,497517,+,FBXO25,LATENT,True,T,477967,497517,0,"0,255,0"
2,FBXO25_2,31.702551,2.032250,0.520938,3.901139,9.574127e-05,0.000816,chr8,497518,522967,+,FBXO25,LATENT,True,T,497518,522967,0,"0,255,0"
4,FBXO25_gene,1868.762014,-0.714125,0.301244,-2.370589,1.775978e-02,0.060028,chr8,406813,477967,+,FBXO25,GENE,False,NA,406813,477967,0,"0,0,255"
6,KBTBD11_1,157.959803,1.992365,0.645662,3.085769,2.030264e-03,0.010462,chr8,2006943,2036493,+,KBTBD11,LATENT,True,T,2006943,2036493,0,"0,255,0"
8,KBTBD11_2,35.493829,3.260711,1.179548,2.764373,5.703225e-03,0.024536,chr8,2036494,2045046,+,KBTBD11,LATENT,True,T,2036494,2045046,0,"0,255,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32876,NCOA5_3,18.049593,-3.034011,1.035915,-2.928822,3.402488e-03,0.017438,chr20,46055991,46058840,-,NCOA5,LATENT,True,C,46055991,46058840,0,"255,0,0"
32879,NCOA5_gene,5135.441007,-0.893668,0.477310,-1.872300,6.116516e-02,0.156732,chr20,46060991,46089962,-,NCOA5,GENE,False,NA,46060991,46089962,0,"0,0,255"
32882,LOC112268269_1,117.772540,-2.554371,0.498424,-5.124898,2.976987e-07,0.000005,chr20,63857315,63861215,-,LOC112268269,LATENT,True,C,63857315,63861215,0,"255,0,0"
32884,LOC112268269_2,100.586113,-1.816507,0.519447,-3.497003,4.705163e-04,0.003316,chr20,63848312,63857314,-,LOC112268269,LATENT,True,C,63848312,63857314,0,"255,0,0"


In [108]:
df_onegene = df_latent[df_latent["name"]=="SYNPO2"]
df_onegene

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE,significant,significant_condition
23106,SYNPO2_1,360.852449,0.458874,0.318528,1.440607,0.149696,0.280750,chr4,119061247,119067897,+,SYNPO2,LATENT,False,NA
23108,SYNPO2_2,857.208836,1.422822,0.625782,2.273671,0.022986,0.068544,chr4,119067898,119073447,+,SYNPO2,LATENT,False,NA
23110,SYNPO2_3,187.487126,1.303177,0.427391,3.049148,0.002295,0.009947,chr4,119073448,119079847,+,SYNPO2,LATENT,True,T
23112,SYNPO2_4,643.160751,1.363043,0.496808,2.743601,0.006077,0.022681,chr4,119079848,119085297,+,SYNPO2,LATENT,True,T
23114,SYNPO2_5,240.987258,0.664281,0.325880,2.038423,0.041508,0.109150,chr4,119085298,119100132,+,SYNPO2,LATENT,False,NA


In [73]:
# Total number of genes with a significant latent region in T or Control or both
sig_genes = list(set(list(df_all["name"])))
len(sig_genes)

4224

In [138]:
df_final = pd.DataFrame()
x = 0
for s in sig_genes:
    df = df_latent[df_latent["name"] == s]
    sig_latent = list(set(list(df["significant_condition"])))
    
    # Check if T and C are sig in the same DoG
    

    df["condition"] = sig_latent[0]

    df["both_conditions_sig"] = False
    if len(sig_latent) > 2:
        df["both_conditions_sig"] = True

    df["all_segment_sig"] = False
    if len(sig_latent) == 1:
        df["all_segment_sig"]  = True        
        
    # Check if the first latent segment is significant
    df["first_segment_sig"] = False
    if df["significant"].iloc[0] == True:
        df["first_segment_sig"] = True
    
    df["last_segment_sig"] = False
    if df["significant"].iloc[-1] == True:
        df["last_segment_sig"] = True    

    df_final = pd.concat([df_final,df])

    x = x+1
    

In [139]:
df_final.head()

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE,significant,significant_condition,condition,both_conditions_sig,all_segment_sig,first_segment_sig,last_segment_sig
20989,BAP1_1,63.072453,-0.605238,0.470062,-1.287571,1.978954e-01,3.597455e-01,chr3,52398754,52401004,-,BAP1,LATENT,False,NA,T,False,False,False,True
20991,BAP1_2,24.395708,1.714610,0.673623,2.545356,1.091664e-02,3.722995e-02,chr3,52396454,52398753,-,BAP1,LATENT,True,T,T,False,False,False,True
20993,BAP1_3,107.905540,5.604757,0.846408,6.621819,3.548063e-11,1.504970e-09,chr3,52381004,52396453,-,BAP1,LATENT,True,T,T,False,False,False,True
32059,TXNL1_1,87.685551,1.385522,0.517543,2.677116,7.425885e-03,3.525255e-02,chr18,56585609,56597209,-,TXNL1,LATENT,True,T,T,False,False,True,False
32060,TXNL1_2,25.536920,0.645562,1.012956,0.637305,5.239261e-01,7.183406e-01,chr18,56572659,56585608,-,TXNL1,LATENT,False,NA,T,False,False,True,False


In [140]:
df_onegene = df_final[df_final["name"]=="SYNPO2"]
df_onegene

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE,significant,significant_condition,condition,both_conditions_sig,all_segment_sig,first_segment_sig,last_segment_sig
23106,SYNPO2_1,360.852449,0.458874,0.318528,1.440607,0.149696,0.280750,chr4,119061247,119067897,+,SYNPO2,LATENT,False,NA,T,False,False,False,False
23108,SYNPO2_2,857.208836,1.422822,0.625782,2.273671,0.022986,0.068544,chr4,119067898,119073447,+,SYNPO2,LATENT,False,NA,T,False,False,False,False
23110,SYNPO2_3,187.487126,1.303177,0.427391,3.049148,0.002295,0.009947,chr4,119073448,119079847,+,SYNPO2,LATENT,True,T,T,False,False,False,False
23112,SYNPO2_4,643.160751,1.363043,0.496808,2.743601,0.006077,0.022681,chr4,119079848,119085297,+,SYNPO2,LATENT,True,T,T,False,False,False,False
23114,SYNPO2_5,240.987258,0.664281,0.325880,2.038423,0.041508,0.109150,chr4,119085298,119100132,+,SYNPO2,LATENT,False,NA,T,False,False,False,False


In [141]:
df2 = df_final.drop_duplicates(subset=['name'])
df2.head()

,name_latent_state,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,chr,start,end,strand,name,LATENT_OR_GENE,significant,significant_condition,condition,both_conditions_sig,all_segment_sig,first_segment_sig,last_segment_sig
20989,BAP1_1,63.072453,-0.605238,0.470062,-1.287571,0.197895,0.359746,chr3,52398754,52401004,-,BAP1,LATENT,False,NA,T,False,False,False,True
32059,TXNL1_1,87.685551,1.385522,0.517543,2.677116,0.007426,0.035253,chr18,56585609,56597209,-,TXNL1,LATENT,True,T,T,False,False,True,False
23709,BEND4_1,1571.496196,0.564640,0.255890,2.206570,0.027344,0.079165,chr4,42096015,42110853,-,BEND4,LATENT,False,NA,T,False,False,False,True
32417,DDX27_1,213.364285,0.728971,0.358153,2.035362,0.041814,0.118606,chr20,49244073,49249723,+,DDX27,LATENT,False,NA,T,False,False,False,True
30838,OSBPL5_1,57.528130,3.522592,1.033814,3.407375,0.000656,0.004666,chr11,3084107,3087107,-,OSBPL5,LATENT,True,T,T,False,False,True,False


In [164]:
df_both_con = df2[df2["both_conditions_sig"] == True]
len(df_both_con)
len(df2)

4224

In [166]:
df3 = df2[df2["both_conditions_sig"] == False]
df_first_seg_not_sig = df3[df3["first_segment_sig"] == False]
df_last_seg_not_sig = df3[df3["last_segment_sig"] == False]
df_all_seg_sig = df3[df3["all_segment_sig"] == True]

def printCounts(df, col):
    print("df_num ", col, len(df))
    dft = df[df["condition"] == "T"]
    dfc = df[df["condition"] == "C"]
    print("df_t ", len(dft))  
    print("df_c ", len(dfc))      
    

    dft_col = df[df[col] == True]
    dfc_col = df[df[col] == False]  
    print("df_col_true ", len(dft_col))  
    print("df_col_false ", len(dfc_col))      
    
    
     
    dftt = dft[dft[col] == True]
    dftf = dft[dft[col] == False]
    
    print("df_tt ", len(dftt))  
    print("df_tf ", len(dftf))      
    
    dfct = dfc[dfc[col] == True]
    dfcf = dfc[dfc[col] == False] 
    print("df_ct ", len(dfct))  
    print("df_cf", len(dfcf))       
    
    
printCounts(df3,"all_segment_sig")
printCounts(df3,"first_segment_sig")
printCounts(df3,"last_segment_sig")
    

df_num  all_segment_sig 4171
df_t  3262
df_c  909
df_col_true  1392
df_col_false  2779
df_tt  1014
df_tf  2248
df_ct  378
df_cf 531
df_num  first_segment_sig 4171
df_t  3262
df_c  909
df_col_true  2116
df_col_false  2055
df_tt  1375
df_tf  1887
df_ct  741
df_cf 168
df_num  last_segment_sig 4171
df_t  3262
df_c  909
df_col_true  3121
df_col_false  1050
df_tt  2651
df_tf  611
df_ct  470
df_cf 439


In [159]:
# import libraries
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

sns.set_style('white')
sns.set_context("paper", font_scale = 4)
plt.figure(figsize=(14, 14))
# g = sns.FacetGrid(df3, col="both_conditions_sig", height=4, aspect=.5)
# g.map(sns.barplot, "first_segment_sig", "total_bill", order=[True, False])


sns.countplot(x="all_segment_sig", hue="condition", data=df3)
plt.savefig('all_segment_sig.png', dpi=300)
plt.clf()

sns.countplot(x="first_segment_sig", hue="condition", data=df3)
plt.savefig('first_segment_sig.png', dpi=300)
plt.clf()

sns.countplot(x="last_segment_sig", hue="condition", data=df3)
plt.savefig('last_segment_sig.png', dpi=300)
plt.clf()

# ax = sns.countplot(x="last_segment_sig", hue="condition", data=df3)






# g = sns.catplot(x="first_segment_sig", hue="condition", col="last_segment_sig",
#                 data=df2, kind="count",# order=["C", "T"],
#                 height=4, aspect=.7);

# g = sns.catplot(x="class", hue="who", col="survived",
#                 data=titanic, kind="count",
#                 height=4, aspect=.7);

<Figure size 1008x1008 with 0 Axes>

In [ ]:



sns.set(style="darkgrid")

# set the figure size
plt.figure(figsize=(14, 14))

# top bar -> sum all values(smoker=No and smoker=Yes) to find y position of the bars
total = len(df3)

# bar chart 1 -> top bars (group of 'smoker=No')
bar1 = sns.barplot(x="df3["first_segment_sig"]",  y="total_bill", data=total, color='darkblue')

# bottom bar ->  take only smoker=Yes values from the data
smoker = tips[tips.smoker=='Yes']

# bar chart 2 -> bottom bars (group of 'smoker=Yes')
bar2 = sns.barplot(x="day", y="total_bill", data=smoker, estimator=sum, ci=None,  color='lightblue')

# add legend
top_bar = mpatches.Patch(color='darkblue', label='smoker = No')
bottom_bar = mpatches.Patch(color='lightblue', label='smoker = Yes')
plt.legend(handles=[top_bar, bottom_bar])

# show the graph
plt.show()

In [75]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='MaDDoG algorithm')
        parser.add_argument('-outdir', action= 'store', metavar='outdir')
        
#export
if __name__=="__main__":
    args = parse_arguments()
    outdir = args.outdir

    

AttributeError: 'NoneType' object has no attribute 'outdir'